# Functionality of this notebook

1. stk is short for single target kluver

In [1]:
import os, sys
import scipy.io as sio
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

""" 
    user defined module
"""

# import_nbmodule used for import package in .ipynb
import import_nbmodule

# add path '../'
sys.path.append(os.path.abspath(os.path.join('..')))

# import ciCoherence_overtime in ../connAnalyTool/synchronization_indices.ipynb
from connAnalyTool.synchronization_indices import ciCoherence_acrosstrials
from connAnalyTool.synchronization_indices import ciCoherence_overtime


# import animal_folder in ../util/folder_extract.pynb 
from util.folder_extract import animal_folder

importing Jupyter notebook from /home/lingling/yang7003@umn.edu/NMRC_umn/Codes/FCAnalysis/connAnalyTool/synchronization_indices.ipynb
importing Jupyter notebook from /home/lingling/yang7003@umn.edu/NMRC_umn/Codes/FCAnalysis/util/folder_extract.ipynb


In [2]:
animal = 'Pinky'

### function ciCOH1file 
1. calculate ciCOH of each pair of channels in return phase for one file

2. only analyze the return phase whose duration is longer than 500ms and shorter than 1s

In [3]:
def ciCOH1file(file):
    
    """
        return the ciCOH in the return phase for a file containing a few trials
        
        Returns:
            ciCOH_1file: corrected imaginary coherence value of all pairs of channels for all trials in file 
                         (nchns * nchns * ntrials)
    """
    

    ######################################
    #
    #    extract data of lfp, idxevent, fs
    #
    ######################################
    
    # load the stk trial data in beta1 band with idxevent, idxevent_varNames, fs
    matdat = sio.loadmat(file, variable_names = ['lfptrial_beta1', 'idxevent', 'idxevent_varNames', 'fs'], 
                         struct_as_record = False, squeeze_me = True)

    # lfp trial data: nchns * n_times * n_trials, fs: sample rate
    lfp, fs = matdat['lfptrial_beta1'], matdat['fs'] 

    # idxevent (n_trials * 5): indices for targetonset, reachonset, reach, return and mouth
    idxevent, idxevent_varNames = matdat['idxevent'], matdat['idxevent_varNames']



    #####################################################################
    #
    #    extract column index for event return [ReturnTimeix MouthTimeix]
    #
    #####################################################################
    
    # column index for event return
    coli_return = [i for i, s in enumerate(idxevent_varNames) if 'ReturnTimeix' in s]

    # column index for event Touch
    coli_mouth = [i for i, s in enumerate(idxevent_varNames) if 'MouthTimeix' in s]

    # column indices of return phase 
    cols_phasereturn = coli_return + coli_mouth

    cols = cols_phasereturn



    ###################################################
    #
    #    extract lfp data of return phase for each trial
    #
    ###################################################
    
    # minimal and maximal duration for return phase
    tdur_min, tdur_max = 0.5, 1
    idxdur_min, idxdur_max = int(np.round(tdur_min * fs)), int(np.round(tdur_max * fs))

    # initial ciCOH array for all trials
    nchns,_ ,ntrials = lfp.shape
    ciCOH_1file = np.zeros((nchns, nchns, ntrials))

    for triali in range(ntrials):

        ### extract the extract lfp data of return phase for each trial, only extract that last longer than 500ms ###
        
        # start and end indices over time of specific phase for triali 
        idxtime_str, idxtime_end = int(idxevent[triali, cols[0]]), int(idxevent[triali, cols[1]])

        # remove trials of return phase longer than tdur_max and shorter than tdur_min
        if idxtime_end - idxtime_str < idxdur_min or idxtime_end - idxtime_str > idxdur_max:
            continue;

        # extract lfp data in specific phase of one trial (n_chns * n_times)
        lfp_1trial = lfp[:, idxtime_str:idxtime_end, triali]


        ### calculate ciCOH between each pair of channels ###
        
        for chni in range(nchns-1):

            # signal of channel chni
            signal1 = lfp_1trial[chni, :]

            for chnj in range(chni+1, nchns):

                # signal of channel chnij
                signal2 = lfp_1trial[chnj, :]

                # ciCOH_1file assignment
                ciCOH_1file[chni, chnj, triali] =  ciCoherence_overtime(signal1, signal2)

                del signal2
            del signal1

        del idxtime_str, idxtime_end, lfp_1trial
        
        
    # return
    return ciCOH_1file

### load path of stk trial data in beta band 

In [4]:
# extract the data, result and metainf folder of animal
anidatfolder, aniresfolder, animetafolder = animal_folder(animal = animal)

# load data folder
loaddatafolder = os.path.join(anidatfolder, 'epochs_filtered', 'betaBand')

# glob all .mat files (both mild and normal files)
files = glob.glob(os.path.join(loaddatafolder, '*.mat'))

### extract ciCOH_normal and ciCOH_mild from all files

In [5]:
# delete ciCOH_mild if exist
if 'ciCOH_mild' in locals():
    del ciCOH_mild

# delete ciCOH_normal if exist
if 'ciCOH_normal' in locals():
    del ciCOH_normal
    
for file in files:

    # ciCOH value of all pairs of channels for all trials in one file (nchns * nchns * ntrials)
    ciCOH_1file = ciCOH1file(file)
    
    # normal case
    if 'normal' in file and 'mild' not in file:
        if 'ciCOH_normal' not in locals():
            # first normal file
            ciCOH_normal = ciCOH_1file
        else:
            # append normal file
            ciCOH_normal = np.concatenate((ciCOH_normal, ciCOH_1file), axis=2)
    
    
    # mild case
    if 'mild' in file and 'normal' not in file:
        if 'ciCOH_mild' not in locals():
            # first mild file
            ciCOH_mild = ciCOH_1file
        else:
            # append mild file
            ciCOH_mild = np.concatenate((ciCOH_mild, ciCOH_1file), axis=2)
            
    del ciCOH_1file

### save calculated ciCOH

In [ ]:
import pickle

# store ciCOH_normal and ciCOH_mild into dict res_ciCOH
res_ciCOH['normal'], res_ciCOH['mild'] = ciCOH_normal, ciCOH_mild

# file storing the ciCOH results
ciCOHfile = os.path.join(aniresfolder, 'res_ciCOH.pickle')

pickle.dump(res_ciCOH, open(ciCOHfile, "wb" ) )

### plot section

In [ ]:
plt.imshow(np.mean(ciCOH_mild, axis = 2));
plt.colorbar()
plt.title('mild')
plt.show()

In [ ]:
plt.imshow(np.mean(ciCOH_normal, axis = 2))
plt.title('normal')
plt.colorbar()
plt.show()

In [ ]:
print(np.mean(ciCOH_normal, axis = 2))

In [ ]:
print(np.mean(ciCOH_mild, axis = 2))